In [24]:
import requests
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
from joblib import Parallel, delayed
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [25]:
def extract_links(ref_begin, page_count):
    links = []

    for i in tqdm(range(1, page_count + 1)):
        ref = f'{ref_begin}?page={i}'
        driver.get(ref)
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        tree = BeautifulSoup(driver.page_source, 'html.parser')

        elems = tree.find('div', {'class' : 'ProductList_products-list__We3gK'}).find_all('a', {'data-qa-article':True})

        for elem in elems:
            links.append('https://sokolov.ru/' + elem.get('href'))
            
    return links

def extract_links_2(ref_begin, page_count):
    links = []
    
    for i in tqdm(range(1, 147)):
        ref = f'{ref_begin}?page={i}'
        driver.get(ref)
        time.sleep(2)
        bs = BeautifulSoup(driver.page_source)
        rings = bs.find_all('a',attrs={'class':'ProductListItem_product-link__T8HTy'})
        links.extend([('https://sokolov.ru' + ring.get('href'), i) for ring in rings])
    driver.close
    return links

In [29]:
driver = webdriver.Chrome()
extracted_ring_links = extract_links_2('https://sokolov.ru/jewelry-catalog/rings/', 146)

ring_links = open('ring_links.txt', 'w')
for link in extracted_ring_links:
    ring_links.write(str(link[0]) + ' ' + str(link[1]))
    ring_links.write('\n')
ring_links.close()

100%|████████████████████████████████████████████████████████████████████████████████| 146/146 [08:44<00:00,  3.59s/it]


In [30]:
def extract_data_threaded(ref):
    chrome_options = Options()
    chrome_options.add_argument("--headless") 
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(ref)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    try:
        tree = BeautifulSoup(driver.page_source, 'html.parser')

        chars_table = tree.find('div', {'class' : "Characteristics_sklv-characteristics__fdRmP"})
        chars_names = chars_table.find_all('div', {'class' : 'Characteristics_sklv-product-page-characteristics-row-name__wa7Gg'})
        chars_values = chars_table.find_all('div', {'class' : 'Characteristics_sklv-product-page-characteristics-row-vals__NwNL9'})

        rings_dict = {}

        for i in range(len(chars_names)):
            name = chars_names[i].span.get_text()
            value = chars_values[i].span.get_text()

            #генератор имени для признаков с повторяющимся названием
            if name in rings_dict.keys():
                i = 2
                while (name + f' {i}') in rings_dict.keys():
                    i += 1
                name += f' {i}'

            rings_dict[name] = value

        rings_list.append(rings_dict)
        driver.close()
    except:
        problem_links.append(ref)
        driver.close()
        
def extract_data(ref, driver, p):
    driver.get(ref)
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    try:
        tree = BeautifulSoup(driver.page_source, 'html.parser')

        chars_table = tree.find('div', {'class' : "Characteristics_sklv-characteristics__fdRmP"})
        chars_names = chars_table.find_all('div', {'class' : 'Characteristics_sklv-product-page-characteristics-row-name__wa7Gg'})
        chars_values = chars_table.find_all('div', {'class' : 'Characteristics_sklv-product-page-characteristics-row-vals__NwNL9'})
        rings_dict = {}

        for i in range(len(chars_names)):
            name = chars_names[i].span.get_text()
            value = chars_values[i].span.get_text()

            #генератор имени для признаков с повторяющимся названием
            if name in rings_dict.keys():
                i = 2
                while (name + f' {i}') in rings_dict.keys():
                    i += 1
                name += f' {i}'

            rings_dict[name] = value

        rings_dict['Цена со скидкой'] = tree.find('div', {'class': 'ProductPrice_sklv-price__new__hdyUa'}).get('data-detail-price')
        try:
            rings_dict['Цена без скидки'] = tree.find('div', {'class': 'ProductPrice_sklv-price__old__QlDQl'}).get('data-old-price')
        except:
            rings_dict['Цена без скидки'] = rings_dict['Цена со скидкой']
        
        rings_dict['Номер страницы'] = p

        rings_list.append(rings_dict)
        
    except:
        problem_links.append(ref)

In [31]:
# rings_list = []
# problem_links = []

# nj = -1
# Parallel(n_jobs=nj, prefer = 'threads')(
#     delayed(extract_data_threaded)(ref)
#     for ref in tqdm(extracted_ring_links))

In [32]:
extracted_ring_links = []

links_from_file = open('ring_links.txt', 'r')
for line in links_from_file:
    extracted_ring_links.append(line.strip().split())

In [33]:
len(extracted_ring_links)

10476

In [51]:
driver = webdriver.Chrome()
rings_list = []
problem_links = []

for i in tqdm(range(2414, len(extracted_ring_links))):
    extract_data(extracted_ring_links[i][0], driver, extracted_ring_links[i][1])

driver.close()

100%|████████████████████████████████████████████████████████████████████████████| 8062/8062 [5:40:47<00:00,  2.54s/it]


In [50]:
rings_list, problem_links

([{'Артикул': '93011090',
   'Бренд': 'SOKOLOV',
   'Коллекция': 'RUSSE',
   'Для кого': 'Для женщин',
   'Примерный вес': '2.54 г',
   'Тип металла': 'Золочёное серебро',
   'Проба': '925',
   'Покрытие': 'Золочение',
   'Тип вставки': 'Фианит',
   'Форма вставки': 'Сердце',
   'Количество': '1 шт',
   'Цвет': 'Зелёный',
   'Вес': '0.7 карат',
   'Размеры вставки': '6 мм * 6 мм',
   'Ширина кольца': '2 мм',
   'Ширина': '7 мм',
   'Высота': '8 мм',
   'Цена со скидкой': '3150',
   'Цена без скидки': '6990',
   'Номер страницы': '34'},
  {'Артикул': '93011094',
   'Бренд': 'SOKOLOV',
   'Для кого': 'Для женщин',
   'Примерный вес': '1 г',
   'Тип металла': 'Золочёное серебро',
   'Проба': '925',
   'Покрытие': 'Золочение',
   'Тип вставки': 'Фианит',
   'Форма вставки': 'Круг',
   'Количество': '1 шт',
   'Цвет': 'Бесцветный',
   'Вес': '0.297 карат',
   'Размеры вставки': '3, 5 мм',
   'Цена со скидкой': '1170',
   'Цена без скидки': '2590',
   'Номер страницы': '34'},
  {'Артикул': '

In [52]:
len(problem_links)

44

In [43]:
problem_links = ['https://sokolov.ru/jewelry-catalog/product/1011542/',
  'https://sokolov.ru/jewelry-catalog/product/1012159/',
  'https://sokolov.ru/jewelry-catalog/product/1012355/',
  'https://sokolov.ru/jewelry-catalog/product/7010080/',
  'https://sokolov.ru/jewelry-catalog/product/1011656/',
  'https://sokolov.ru/jewelry-catalog/product/1011277/',
  'https://sokolov.ru/jewelry-catalog/product/1012308/',
  'https://sokolov.ru/jewelry-catalog/product/1012023/',
  'https://sokolov.ru/jewelry-catalog/product/019303/',
  'https://sokolov.ru/jewelry-catalog/product/1110218/',
  'https://sokolov.ru/jewelry-catalog/product/1110211/',
  'https://sokolov.ru/jewelry-catalog/product/110187/',
  'https://sokolov.ru/jewelry-catalog/product/1110216-3/',
  'https://sokolov.ru/jewelry-catalog/product/94013245/',
  'https://sokolov.ru/jewelry-catalog/product/6015116/',
  'https://sokolov.ru/jewelry-catalog/product/1110222/',
  'https://sokolov.ru/jewelry-catalog/product/791267/',
  'https://sokolov.ru/jewelry-catalog/product/94010426/',
  'https://sokolov.ru/jewelry-catalog/product/1011495/',
  'https://sokolov.ru/jewelry-catalog/product/87010066/',
  'https://sokolov.ru/jewelry-catalog/product/019188/',
  'https://sokolov.ru/jewelry-catalog/product/791239/',
  'https://sokolov.ru/jewelry-catalog/product/83010104/',
  'https://sokolov.ru/jewelry-catalog/product/87010077/',
  'https://sokolov.ru/jewelry-catalog/product/87010081/',
  'https://sokolov.ru/jewelry-catalog/product/92011741/',
  'https://sokolov.ru/jewelry-catalog/product/92011743/',
  'https://sokolov.ru/jewelry-catalog/product/92011747/',
  'https://sokolov.ru/jewelry-catalog/product/94014052/',
  'https://sokolov.ru/jewelry-catalog/product/019240/',
  'https://sokolov.ru/jewelry-catalog/product/1012593-3/',
  'https://sokolov.ru/jewelry-catalog/product/1114006-13/',
  'https://sokolov.ru/jewelry-catalog/product/71-00078/',
  'https://sokolov.ru/jewelry-catalog/product/71-00093/',
  'https://sokolov.ru/jewelry-catalog/product/71-00103/',
  'https://sokolov.ru/jewelry-catalog/product/71-00232/',
  'https://sokolov.ru/jewelry-catalog/product/71-00251/',
  'https://sokolov.ru/jewelry-catalog/product/712391/',
  'https://sokolov.ru/jewelry-catalog/product/714252/',
  'https://sokolov.ru/jewelry-catalog/product/714646/',
  'https://sokolov.ru/jewelry-catalog/product/714657/',
  'https://sokolov.ru/jewelry-catalog/product/714675/',
  'https://sokolov.ru/jewelry-catalog/product/714756/',
  'https://sokolov.ru/jewelry-catalog/product/714802/',
  'https://sokolov.ru/jewelry-catalog/product/714830/',
  'https://sokolov.ru/jewelry-catalog/product/715740/',
  'https://sokolov.ru/jewelry-catalog/product/716297/',
  'https://sokolov.ru/jewelry-catalog/product/716519/',
  'https://sokolov.ru/jewelry-catalog/product/716597/',
  'https://sokolov.ru/jewelry-catalog/product/018392-2/',
  'https://sokolov.ru/jewelry-catalog/product/2011214/',
  'https://sokolov.ru/jewelry-catalog/product/6014210/',
  'https://sokolov.ru/jewelry-catalog/product/018626/',
  'https://sokolov.ru/jewelry-catalog/product/6015001/',
  'https://sokolov.ru/jewelry-catalog/product/6015009/',
  'https://sokolov.ru/jewelry-catalog/product/6015010/',
  'https://sokolov.ru/jewelry-catalog/product/6015011/',
  'https://sokolov.ru/jewelry-catalog/product/6015013/',
  'https://sokolov.ru/jewelry-catalog/product/6015079/',
  'https://sokolov.ru/jewelry-catalog/product/6015088/',
  'https://sokolov.ru/jewelry-catalog/product/6015089/',
  'https://sokolov.ru/jewelry-catalog/product/6015114/',
  'https://sokolov.ru/jewelry-catalog/product/6015114-3/',
  'https://sokolov.ru/jewelry-catalog/product/6015115/',
  'https://sokolov.ru/jewelry-catalog/product/6015115-3/',
  'https://sokolov.ru/jewelry-catalog/product/6015118/',
  'https://sokolov.ru/jewelry-catalog/product/6015119/',
  'https://sokolov.ru/jewelry-catalog/product/7010041/',
  'https://sokolov.ru/jewelry-catalog/product/7010053/',
  'https://sokolov.ru/jewelry-catalog/product/7010054/',
  'https://sokolov.ru/jewelry-catalog/product/7010068/',
  'https://sokolov.ru/jewelry-catalog/product/7010094-3/',
  'https://sokolov.ru/jewelry-catalog/product/7010095-3/',
  'https://sokolov.ru/jewelry-catalog/product/71-00003/',
  'https://sokolov.ru/jewelry-catalog/product/71-00005/',
  'https://sokolov.ru/jewelry-catalog/product/71-00007/',
  'https://sokolov.ru/jewelry-catalog/product/71-00019/',
  'https://sokolov.ru/jewelry-catalog/product/71-00100-3/',
  'https://sokolov.ru/jewelry-catalog/product/714227/',
  'https://sokolov.ru/jewelry-catalog/product/714261/',
  'https://sokolov.ru/jewelry-catalog/product/714318/',
  'https://sokolov.ru/jewelry-catalog/product/714346/',
  'https://sokolov.ru/jewelry-catalog/product/714434/',
  'https://sokolov.ru/jewelry-catalog/product/714609/',
  'https://sokolov.ru/jewelry-catalog/product/714611/',
  'https://sokolov.ru/jewelry-catalog/product/714648/',
  'https://sokolov.ru/jewelry-catalog/product/714672/',
  'https://sokolov.ru/jewelry-catalog/product/714739/',
  'https://sokolov.ru/jewelry-catalog/product/714842/',
  'https://sokolov.ru/jewelry-catalog/product/714845/',
  'https://sokolov.ru/jewelry-catalog/product/714973/',
  'https://sokolov.ru/jewelry-catalog/product/715005/',
  'https://sokolov.ru/jewelry-catalog/product/715081/',
  'https://sokolov.ru/jewelry-catalog/product/715238/',
  'https://sokolov.ru/jewelry-catalog/product/715589/',
  'https://sokolov.ru/jewelry-catalog/product/715592/',
  'https://sokolov.ru/jewelry-catalog/product/715664/',
  'https://sokolov.ru/jewelry-catalog/product/715739/',
  'https://sokolov.ru/jewelry-catalog/product/715741/',
  'https://sokolov.ru/jewelry-catalog/product/715817/',
  'https://sokolov.ru/jewelry-catalog/product/715857/',
  'https://sokolov.ru/jewelry-catalog/product/715875/',
  'https://sokolov.ru/jewelry-catalog/product/715882/',
  'https://sokolov.ru/jewelry-catalog/product/715896/',
  'https://sokolov.ru/jewelry-catalog/product/715897/',
  'https://sokolov.ru/jewelry-catalog/product/715920/',
  'https://sokolov.ru/jewelry-catalog/product/715986/',
  'https://sokolov.ru/jewelry-catalog/product/716027/',
  'https://sokolov.ru/jewelry-catalog/product/716077/',
  'https://sokolov.ru/jewelry-catalog/product/716090/',
  'https://sokolov.ru/jewelry-catalog/product/716107/',
  'https://sokolov.ru/jewelry-catalog/product/716117/',
  'https://sokolov.ru/jewelry-catalog/product/716125/',
  'https://sokolov.ru/jewelry-catalog/product/716149/',
  'https://sokolov.ru/jewelry-catalog/product/716161/',
  'https://sokolov.ru/jewelry-catalog/product/716164/',
  'https://sokolov.ru/jewelry-catalog/product/716165/',
  'https://sokolov.ru/jewelry-catalog/product/716168/',
  'https://sokolov.ru/jewelry-catalog/product/716172/',
  'https://sokolov.ru/jewelry-catalog/product/716173/',
  'https://sokolov.ru/jewelry-catalog/product/716174/',
  'https://sokolov.ru/jewelry-catalog/product/716175/',
  'https://sokolov.ru/jewelry-catalog/product/716184/',
  'https://sokolov.ru/jewelry-catalog/product/716192/',
  'https://sokolov.ru/jewelry-catalog/product/716199/',
  'https://sokolov.ru/jewelry-catalog/product/716213/',
  'https://sokolov.ru/jewelry-catalog/product/716288/',
  'https://sokolov.ru/jewelry-catalog/product/716340/',
  'https://sokolov.ru/jewelry-catalog/product/716463/',
  'https://sokolov.ru/jewelry-catalog/product/716479/',
  'https://sokolov.ru/jewelry-catalog/product/716491/',
  'https://sokolov.ru/jewelry-catalog/product/716493/',
  'https://sokolov.ru/jewelry-catalog/product/716493/',
  'https://sokolov.ru/jewelry-catalog/product/716505/',
  'https://sokolov.ru/jewelry-catalog/product/716515/',
  'https://sokolov.ru/jewelry-catalog/product/716565/',
  'https://sokolov.ru/jewelry-catalog/product/716566/',
  'https://sokolov.ru/jewelry-catalog/product/716567/',
  'https://sokolov.ru/jewelry-catalog/product/716606/',
  'https://sokolov.ru/jewelry-catalog/product/716606-3/',
  'https://sokolov.ru/jewelry-catalog/product/716608/',
  'https://sokolov.ru/jewelry-catalog/product/716646/',
  'https://sokolov.ru/jewelry-catalog/product/716647/',
  'https://sokolov.ru/jewelry-catalog/product/716724/',
  'https://sokolov.ru/jewelry-catalog/product/716734/',
  'https://sokolov.ru/jewelry-catalog/product/716735/',
  'https://sokolov.ru/jewelry-catalog/product/716770/',
  'https://sokolov.ru/jewelry-catalog/product/716821/',
  'https://sokolov.ru/jewelry-catalog/product/716906/',
  'https://sokolov.ru/jewelry-catalog/product/716928/',
  'https://sokolov.ru/jewelry-catalog/product/716939/',
  'https://sokolov.ru/jewelry-catalog/product/791010/',
  'https://sokolov.ru/jewelry-catalog/product/791015/',
  'https://sokolov.ru/jewelry-catalog/product/791034/',
  'https://sokolov.ru/jewelry-catalog/product/791160/',
  'https://sokolov.ru/jewelry-catalog/product/791171/',
  'https://sokolov.ru/jewelry-catalog/product/791204/',
  'https://sokolov.ru/jewelry-catalog/product/791212/',
  'https://sokolov.ru/jewelry-catalog/product/791216-3/',
  'https://sokolov.ru/jewelry-catalog/product/791245/',
  'https://sokolov.ru/jewelry-catalog/product/83010037/',
  'https://sokolov.ru/jewelry-catalog/product/83010059/',
  'https://sokolov.ru/jewelry-catalog/product/83010063/',
  'https://sokolov.ru/jewelry-catalog/product/83010064/',
  'https://sokolov.ru/jewelry-catalog/product/83010069/',
  'https://sokolov.ru/jewelry-catalog/product/83010073/',
  'https://sokolov.ru/jewelry-catalog/product/84010024/',
  'https://sokolov.ru/jewelry-catalog/product/87010002/',
  'https://sokolov.ru/jewelry-catalog/product/87010012/',
  'https://sokolov.ru/jewelry-catalog/product/87010015/',
  'https://sokolov.ru/jewelry-catalog/product/87010016/',
  'https://sokolov.ru/jewelry-catalog/product/87010024/',
  'https://sokolov.ru/jewelry-catalog/product/87010025/',
  'https://sokolov.ru/jewelry-catalog/product/87010026/',
  'https://sokolov.ru/jewelry-catalog/product/87010028/',
  'https://sokolov.ru/jewelry-catalog/product/87010029/',
  'https://sokolov.ru/jewelry-catalog/product/87010030/',
  'https://sokolov.ru/jewelry-catalog/product/87010032/',
  'https://sokolov.ru/jewelry-catalog/product/87010036/',
  'https://sokolov.ru/jewelry-catalog/product/87010041/',
  'https://sokolov.ru/jewelry-catalog/product/87010042/',
  'https://sokolov.ru/jewelry-catalog/product/87010045/',
  'https://sokolov.ru/jewelry-catalog/product/87010047/',
  'https://sokolov.ru/jewelry-catalog/product/87010093/',
  'https://sokolov.ru/jewelry-catalog/product/92011502/',
  'https://sokolov.ru/jewelry-catalog/product/92011503/',
  'https://sokolov.ru/jewelry-catalog/product/92011546/',
  'https://sokolov.ru/jewelry-catalog/product/92011591/',
  'https://sokolov.ru/jewelry-catalog/product/92011592/',
  'https://sokolov.ru/jewelry-catalog/product/92011593/',
  'https://sokolov.ru/jewelry-catalog/product/92011594/',
  'https://sokolov.ru/jewelry-catalog/product/92011798/',
  'https://sokolov.ru/jewelry-catalog/product/92011814/',
  'https://sokolov.ru/jewelry-catalog/product/92011816/',
  'https://sokolov.ru/jewelry-catalog/product/92011964/',
  'https://sokolov.ru/jewelry-catalog/product/92011966/',
  'https://sokolov.ru/jewelry-catalog/product/92012030/',
  'https://sokolov.ru/jewelry-catalog/product/92012090/',
  'https://sokolov.ru/jewelry-catalog/product/92012091/',
  'https://sokolov.ru/jewelry-catalog/product/92012092/',
  'https://sokolov.ru/jewelry-catalog/product/92012098/',
  'https://sokolov.ru/jewelry-catalog/product/92012103/',
  'https://sokolov.ru/jewelry-catalog/product/92012104/',
  'https://sokolov.ru/jewelry-catalog/product/92012140/',
  'https://sokolov.ru/jewelry-catalog/product/92012157/',
  'https://sokolov.ru/jewelry-catalog/product/92012163/',
  'https://sokolov.ru/jewelry-catalog/product/92014274/',
  'https://sokolov.ru/jewelry-catalog/product/92014323/',
  'https://sokolov.ru/jewelry-catalog/product/93010021/',
  'https://sokolov.ru/jewelry-catalog/product/93010163/',
  'https://sokolov.ru/jewelry-catalog/product/93010409/',
  'https://sokolov.ru/jewelry-catalog/product/93010524/',
  'https://sokolov.ru/jewelry-catalog/product/93010600/',
  'https://sokolov.ru/jewelry-catalog/product/93010871/',
  'https://sokolov.ru/jewelry-catalog/product/93011082/',
  'https://sokolov.ru/jewelry-catalog/product/93011088/']

In [53]:
problem_links_file = open('problem_links2.txt', 'w')
for link in problem_links:
    problem_links_file.write(link)
    problem_links_file.write('\n')
problem_links_file.close()

In [54]:
df = pd.DataFrame(rings_list)
df

,Артикул,Бренд,Коллекция,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,...,Цветность 9,Чистота 9,Цветность 10,Чистота 10,Тип вставки 14,Форма вставки 14,Количество 14,Цвет 14,Вес 14,Размеры вставки 14
0,93011090,SOKOLOV,RUSSE,Для женщин,2.54 г,Золочёное серебро,925,Золочение,Фианит,Сердце,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93011094,SOKOLOV,NaN,Для женщин,1 г,Золочёное серебро,925,Золочение,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93011095,SOKOLOV,NaN,Для женщин,3.17 г,Золочёное серебро,925,"Золочение, Родирование, Эмаль",Эмаль,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,93011096,SOKOLOV,NaN,Для женщин,2.99 г,Золочёное серебро,925,"Золочение, Эмаль",Эмаль,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,93011100,SOKOLOV,NaN,Для женщин,1.23 г,Золочёное серебро,925,Золочение,Без вставок,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8013,94051161,SOKOLOV,SKLV,Для женщин,4.1 г,Родированное серебро,925,NaN,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8014,95010021,SOKOLOV,NaN,Для мужчин,12.34 г,Чернёное серебро,925,NaN,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8015,95010022,SOKOLOV,NaN,Для мужчин,12.37 г,Чернёное серебро,925,NaN,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8016,95010034,SOKOLOV,NaN,Для мужчин,9.12 г,Чернёное серебро,925,Чернение,Без вставок,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df.to_csv('output2.csv')

In [108]:
df1 = pd.read_csv('output1.csv')
df2 = pd.read_csv('output2.csv')

finaldf = pd.concat([df1, df2])
finaldf.reset_index().drop(['index', 'Unnamed: 0'], axis=1).to_csv('data.csv')